In [10]:
from scapy.all import rdpcap, UDP, IP
import datetime
import re
from tqdm import tqdm

def analyse_experiment(pcap_file, entry_file):

    packets = rdpcap(pcap_file)

    entry_timestamps = []
    packet_timestamps = []

    for pkt in tqdm(packets):
        if IP in pkt and UDP in pkt:
            if pkt[IP].dst == "10.0.0.1":
                ts = pkt.time
                ts = float(pkt.time)
                sec = int(ts)
                nsec = int((ts - sec) * 1_000_000_000)

                formatted = f"{datetime.datetime.fromtimestamp(sec).strftime('%H:%M:%S')}.{nsec:09d}"
                packet_timestamps.append(formatted)

    with open(entry_file) as f:
        lines = f.readlines()
        add_lines = []
        timestamps = []
        for line in tqdm(lines):
            if "Adding entry at:" in line:
                add_lines.append(line.strip())
        for line in tqdm(add_lines):
            match = re.search(r"Adding entry at:\s+(\d{2}:\d{2}:\d{2}\.\d+)", line)
            if match:
                timestamp = match.group(1)
                #print("Extracted timestamp:", timestamp)
                timestamps.append(timestamp)
            else:
                print("No timestamp found")
        #print(timestamps)
        entry_timestamps = timestamps
    def to_ns_timestamp(tstr):
        hh, mm, rest = tstr.split(":")
        ss, ns = rest.split(".")
        hh = int(hh)
        mm = int(mm)
        ss = int(ss)
        ns = int(ns)
        total_ns = ((hh*3600 + mm*60 + ss) * 1_000_000_000) + ns
        return total_ns


    entry_ns = [to_ns_timestamp(t) for t in entry_timestamps]
    packet_ns = [to_ns_timestamp(t) for t in packet_timestamps]

    packet_ns.sort()

    latencies = []

    for entry_time in entry_ns:
        first_packet_after_entry = None

        for packet_time in packet_ns:
            if packet_time > entry_time:
                first_packet_after_entry = packet_time
                break

        if first_packet_after_entry is None:
            latencies.append(None)
            continue

        latency = first_packet_after_entry - entry_time
        latencies.append(latency)

    latencies_cleaned = [l for l in latencies if l is not None]
    print(f"\nDurchschnittliche Latenz: {sum(latencies_cleaned)/len(latencies_cleaned)/1_000:.3f} ns")
    #print(f"Maximale Latenz: {max(latencies_cleaned):.3f} microseconds")
    print(f"Minimale Latenz: {min(latencies_cleaned):.3f} microseconds, {min(latencies_cleaned)/1_000:.3f} ns")

In [11]:
analyse_experiment("measurements/test1.pcap", "measurements/entries_1.txt")

100%|██████████| 89/89 [00:00<00:00, 639200.44it/s]



Durchschnittliche Latenz: 1857.539 ns
Minimale Latenz: 207612.000 microseconds, 207.612 ns


In [12]:
analyse_experiment("measurements/test2.pcap", "measurements/entries_2.txt")

100%|██████████| 304/304 [00:00<00:00, 1103000.36it/s]



Durchschnittliche Latenz: 2243.204 ns
Minimale Latenz: 99985.000 microseconds, 99.985 ns


In [13]:
analyse_experiment("measurements/test4.pcap", "measurements/entries_4.txt")

100%|██████████| 30/30 [00:00<00:00, 347594.25it/s]



Durchschnittliche Latenz: 1736.784 ns
Minimale Latenz: 1098.000 microseconds, 1.098 ns


In [14]:
analyse_experiment("measurements/test5.pcap", "measurements/entries_5.txt")

100%|██████████| 91/91 [00:00<00:00, 786972.50it/s]



Durchschnittliche Latenz: 672.426 ns
Minimale Latenz: 55.000 microseconds, 0.055 ns


In [ ]:
analyse_experiment("measurements/test6.pcap", "measurements/entries_6.txt")

In [ ]:
analyse_experiment("measurements/test7.pcap", "measurements/entries_7.txt")